In [3]:
import numpy as np
from scipy import sparse
import operator

In [4]:
def build_sparse_link_matrix(filename):
    """
    Costruisce la Link Matrix in formato sparso (CSR).
    NON applica fisicamente la patch nella matrice per preservare la sparsità,
    ma identifica i nodi dangling per gestirli nel calcolo vettoriale.
    """
    links = []
    out_degree = {}
    
    try:
        with open(filename, 'r') as file:
            # 1. Lettura Header
            header = file.readline().strip().split()
            if not header: raise ValueError("File vuoto o header mancante")
            N = int(header[0])
            
            # 2. Skip URL mapping
            for _ in range(N): file.readline()
            
            # 3. Lettura Archi
            for line in file:
                parts = line.strip().split()
                if len(parts) == 2:
                    src, tgt = int(parts[0]), int(parts[1])
                    out_degree[src] = out_degree.get(src, 0) + 1
                    links.append((src, tgt))
                    
    except Exception as e:
        print(f"Errore lettura file: {e}")
        return None, 0, None

    # Preparazione dati per matrice sparsa (Data, Row, Col)
    data = []
    rows = [] # Target (i)
    cols = [] # Source (j)
    
    # Identifichiamo i nodi dangling (quelli che non hanno out_degree)
    # Creiamo una maschera booleana: True se il nodo è dangling
    is_dangling = np.ones(N, dtype=bool) 
    
    for src, tgt in links:
        src_idx = src - 1
        tgt_idx = tgt - 1
        
        # Se siamo qui, il nodo src ha almeno un link, quindi non è dangling
        is_dangling[src_idx] = False
        
        # Calcolo valore 1/n_j
        val = 1.0 / out_degree[src]
        
        rows.append(tgt_idx)
        cols.append(src_idx)
        data.append(val)
        
    # Costruzione matrice CSR (Compressed Sparse Row)
    # Nota: Usiamo CSR perché è molto veloce per la moltiplicazione matrice-vettore
    A_sparse = sparse.csr_matrix((data, (rows, cols)), shape=(N, N))
    
    dangling_indices = np.where(is_dangling)[0]
    print(f"Matrice sparsa costruita: {N} nodi, {len(data)} link validi.")
    print(f"Nodi dangling identificati: {len(dangling_indices)} (gestiti implicitamente).")
    
    return A_sparse, N, dangling_indices

In [5]:
def calculate_pagerank_sparse(A_sparse, N, dangling_indices, m=0.15, max_iter=200, tol=1e-7):
    """
    Calcola il PageRank usando operazioni sparse efficienti.
    Gestisce la patch dei dangling nodes dinamicamente.
    """
    
    # Inizializzazione vettore x (1/N)
    x = np.full(N, 1.0/N)
    
    # Vettore costante di teletrasporto (parte fissa m/N)
    teleport_contribution = m / N
    
    # Pre-calcolo indici dangling per accesso veloce (se non passato come array)
    if not isinstance(dangling_indices, np.ndarray):
        dangling_indices = np.array(dangling_indices)
        
    iterations = 0
    
    for k in range(max_iter):
        x_prev = x.copy()
        
        # 1. Moltiplicazione Standard su matrice sparsa
        # Calcola solo i link ESISTENTI. Ignora i dangling nodes per ora (sono righe di zeri).
        # Risultato parziale: flusso di probabilità dai nodi normali.
        Ax = A_sparse.dot(x_prev)
        
        # 2. Gestione "Virtuale" dei Dangling Nodes
        # Calcoliamo quanta probabilità è finita nei vicoli ciechi
        dangling_mass_sum = np.sum(x_prev[dangling_indices])
        
        # Questa massa viene ridistribuita equamente a tutti i nodi (Patch 1/N)
        # Deve essere smorzata dal fattore (1-m)
        dangling_correction = (1 - m) * (dangling_mass_sum / N)
        
        # 3. Combinazione finale
        # x = (flusso dai link * smorzamento) + (flusso dai dangling * smorzamento) + (teletrasporto random)
        x = (1 - m) * Ax + dangling_correction + teleport_contribution
        
        # Check convergenza (Norma L1)
        diff = np.sum(np.abs(x - x_prev))
        iterations = k + 1
        
        if diff < tol:
            break
            
    return x, iterations

In [10]:
# --- PARAMETRI ---
filename = 'hollins.dat'
m = 0.15
top_k = 10

print("--- AVVIO IMPLEMENTAZIONE SPARSA ---")

# 1. Costruzione
A_sparse, N, dangling_nodes = build_sparse_link_matrix(filename)

# 2. Calcolo
if A_sparse is not None:
    pagerank_scores, iters = calculate_pagerank_sparse(A_sparse, N, dangling_nodes, m=m)

    print(f"Calcolo completato in {iters} iterazioni.")

    # 3. Risultati
    page_ids = np.arange(1, N + 1)
    results = list(zip(page_ids, pagerank_scores))
    results_sorted = sorted(results, key=operator.itemgetter(1), reverse=True)

    print(f"\n--- TOP {top_k} Pages (Sparse Method) ---")
    for rank, (page_id, score) in enumerate(results_sorted[:top_k], 1):
        print(f"Rank {rank}: Page ID {page_id} (Score: {score:.6f})")

    # Verifica correttezza matematica (somma deve essere 1)
    print(f"\nSomma totale PageRank: {np.sum(pagerank_scores):.6f} (Atteso: 1.0)")
    
    # For a good evaluation, we print the importance of the least important node, which should approach the theoretical minimum score (m/N).
min_score = results_sorted[-1][1]
expected_min = m / N
print(f"Minimum score (last page): {min_score:.6f}")
print(f"Theoretical minimum score ({m}/N): {expected_min:.6f}")

--- AVVIO IMPLEMENTAZIONE SPARSA ---
Matrice sparsa costruita: 6012 nodi, 23875 link validi.
Nodi dangling identificati: 3189 (gestiti implicitamente).
Calcolo completato in 71 iterazioni.

--- TOP 10 Pages (Sparse Method) ---
Rank 1: Page ID 2 (Score: 0.019879)
Rank 2: Page ID 37 (Score: 0.009288)
Rank 3: Page ID 38 (Score: 0.008610)
Rank 4: Page ID 61 (Score: 0.008065)
Rank 5: Page ID 52 (Score: 0.008027)
Rank 6: Page ID 43 (Score: 0.007165)
Rank 7: Page ID 425 (Score: 0.006583)
Rank 8: Page ID 27 (Score: 0.005989)
Rank 9: Page ID 28 (Score: 0.005572)
Rank 10: Page ID 4023 (Score: 0.004452)

Somma totale PageRank: 1.000000 (Atteso: 1.0)
Minimum score (last page): 0.000058
Theoretical minimum score (0.15/N): 0.000025
